In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from PIL import Image

In [2]:
# 数据增强
# 创建ImageDataGenerator实例
train_datagen = ImageDataGenerator(
    rescale=1./255,       # 将像素值缩放到0-1范围
    shear_range=0.2,      # 随机错切变换
    zoom_range=0.2,       # 随机缩放
    horizontal_flip=True  # 随机水平翻转
)

validation_datagen = ImageDataGenerator(rescale=1./255)  # 验证集只进行缩放

# 使用flow_from_directory方法从目录中加载数据
train_generator = train_datagen.flow_from_directory(
    'data/train',              # 训练数据目录
    target_size=(128, 128),    # 调整图像大小
    batch_size=32,             # 每个批次的图像数
    class_mode='binary'        # 分类模式：二分类
)

validation_generator = validation_datagen.flow_from_directory(
    'data/validation',         # 验证数据目录
    target_size=(128, 128),    # 调整图像大小
    batch_size=32,             # 每个批次的图像数
    class_mode='binary'        # 分类模式：二分类
)

# 数据清洗
for i in os.listdir('data/train/cat'):
    try:
        Image.open(os.path.join('data/train/cat', i))
    except:
        os.remove(os.path.join('data/train/cat', i))
        print(i)

for i in os.listdir('data/train/dog'):
    try:
        Image.open(os.path.join('data/train/dog', i))
    except:
        os.remove(os.path.join('data/train/dog', i))
        print(i)
for i in os.listdir('data/validation/cat'):
    try:
        Image.open(os.path.join('data/validation/cat', i))
    except:
        os.remove(os.path.join('data/validation/cat', i))
        print(i)

for i in os.listdir('data/validation/dog'):
    try:
        Image.open(os.path.join('data/validation/dog', i))
    except:
        os.remove(os.path.join('data/validation/dog', i))
        print(i)


Found 20001 images belonging to 2 classes.
Found 4997 images belonging to 2 classes.


C:\APP\WorkPlace\Anaconda\Lib\site-packages\PIL\TiffImagePlugin.py:870: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [3]:
# 建立模型
model = Sequential()

# 输入层
model.add(Input(shape=(128, 128, 3)))

# 第一个卷积层
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 第二个卷积层
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 第三个卷积层
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 全连接层
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [4]:
#模型训练
model.fit(
    train_generator,
    steps_per_epoch=2000 // 32,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=800 // 32
)


Epoch 1/5


C:\APP\WorkPlace\Anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 291ms/step - accuracy: 0.5113 - loss: 0.7850 - val_accuracy: 0.5175 - val_loss: 0.6949
Epoch 2/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 17s 275ms/step - accuracy: 0.5347 - loss: 0.6938 - val_accuracy: 0.5312 - val_loss: 0.6997
Epoch 3/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 17s 271ms/step - accuracy: 0.5545 - loss: 0.6832 - val_accuracy: 0.6275 - val_loss: 0.6498
Epoch 4/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 17s 268ms/step - accuracy: 0.6098 - loss: 0.6579 - val_accuracy: 0.6263 - val_loss: 0.6365
Epoch 5/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 16s 256ms/step - accuracy: 0.6394 - loss: 0.6411 - val_accuracy: 0.6587 - val_loss: 0.6153


In [5]:
#评估模型
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

#保存模型
model.save('my_model.keras')

157/157 ━━━━━━━━━━━━━━━━━━━━ 10s 64ms/step - accuracy: 0.6782 - loss: 0.6109
Validation Loss: 0.6077237129211426
Validation Accuracy: 0.6772063374519348


In [6]:
#进行模型预测
img_path = "data\cat_dog\cat.0.jpg"
img = image.load_img(img_path, target_size=(128, 128))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

prediction = model.predict(img_array)
if prediction[0] > 0.5:
    print('This is a dog.')
else:
    print('This is a cat.')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
This is a cat.
